## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
from keras.datasets import cifar10
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.resnet50 import ResNet50, preprocess_input
# from keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 256

# 凍結網路層數
# FREEZE_LAYERS = 2

# Epoch 數
NUM_EPOCHS = 100

In [3]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = preprocess_input(x_train) 
x_test = preprocess_input(x_test)
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [9]:
# 以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers


base_model = ResNet50(include_top=False, weights='imagenet',input_shape=(32, 32,3), pooling = 'avg')


for layer in base_model.layers:
    layer.trainable = False
    
x = base_model.output
x = Flatten(name='flatten')(x)
x = Dense(1024, activation = 'relu')(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(10, activation='softmax', name='softmax')(x)
model = Model(inputs=base_model.input, outputs=output_layer)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

In [10]:
# 建立 ImageDataGenerator，並指定我們要做資料增強的數值範圍
data_generator = ImageDataGenerator(
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [12]:
optimizer = Adam(lr=3e-5,decay=0.0001,beta_1=0.99,beta_2=0.999)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model_history = model.fit_generator(data_generator.flow(x_train, y_train, batch_size = BATCH_SIZE),
                                   epochs = NUM_EPOCHS,
                                   validation_data = (x_test, y_test),
                                    steps_per_epoch=x_train.shape[0] // BATCH_SIZE
                                   
                                   )
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/100
195/195 [==============================] - 49s 249ms/step - loss: 2.2111 - acc: 0.2717 - val_loss: 2.5613 - val_acc: 0.4093
Epoch 2/100
195/195 [==============================] - 39s 202ms/step - loss: 1.8355 - acc: 0.3872 - val_loss: 2.4397 - val_acc: 0.4254
Epoch 3/100
195/195 [==============================] - 39s 202ms/step - loss: 1.7389 - acc: 0.4068 - val_loss: 2.3701 - val_acc: 0.4299
Epoch 4/100
195/195 [==============================] - 39s 202ms/step - loss: 1.6784 - acc: 0.4255 - val_loss: 2.2664 - val_acc: 0.4361
Epoch 5/100
195/195 [==============================] - 39s 202ms/step - loss: 1.6478 - acc: 0.4347 - val_loss: 2.3203 - val_acc: 0.4313
Epoch 6/100
195/195 [==============================] - 41s 211ms/step - loss: 1.6199 - acc: 0.4432 - val_loss: 2.2847 - val_acc: 0.4335
Epoch 7/100
195/195 [==============================] - 40s 204ms/step - loss: 1.5944 - acc: 0.4508 - val_loss: 2.2372 - val_acc: 0.4407
Epoch 8/100
195/195 [===========================